In [3]:
# !python3 "/home/lakshmi/notebook/jupyterenv/GSQL_with_SPSA/pymdptoolbox-4.0-b3/src/mdptoolbox/main.py"
!pip install pymdptoolbox

In [2]:
# import mdptoolbox

In [4]:
import numpy as np
import mdptoolbox, mdptoolbox.example
import random
import time

In [5]:
s = 100
a = 5
discount = 0.6
delta=0.6

episodes = 10
iterations = 100000

dql_total_diff= np.zeros((episodes,iterations))
sor_total_diff=np.zeros((episodes,iterations))
sql_total_diff= np.zeros((episodes,iterations))
zapql_total_diff=np.zeros((episodes,iterations))
gsql1_total_diff= np.zeros((episodes,iterations))
gsql2_total_diff=np.zeros((episodes,iterations))
gsql3_total_diff=np.zeros((episodes,iterations))

w=5
w_update_lr=0.1
w_val=[]
w_val.append(w)

for count in range(episodes):
    print(count)
    np.random.seed((count+1)*100)
    random.seed((count+1)*110)
    
    P, R = mdptoolbox.example.rand(s, a)

    time_vi_st=time.time()
    vi = mdptoolbox.mdp.ValueIteration(P, R, discount,epsilon=0.00001)
    vi.run()
    time_vi_end=time.time()

    #SOR QL
    time_sor_original_st=time.time()
    sor = mdptoolbox.mdp.SOR_QLearning_original(P, R, discount,n_iter=iterations)
    sor.run()
    norm_diff_sor = vi.V - sor.q_values
    sor_total_diff[count] = np.linalg.norm(norm_diff_sor,axis =1)
    time_sor_original_end=time.time()

    #Speedy QL
    time_sql_st=time.time()
    ql1 = mdptoolbox.mdp.SpeedyQLearning(P, R, discount,n_iter=iterations)
    ql1.run()
    norm_diff_sql = vi.V - ql1.q_values
    sql_total_diff[count] = np.linalg.norm(norm_diff_sql,axis =1)
    time_sql_end=time.time()
    
    #GSQL1
    time_gsql1_st=time.time()
    ql2 = mdptoolbox.mdp.GSQL1(P, R, discount,n_iter=iterations)
    ql2.run()
    norm_diff_gsql1 = vi.V - ql2.q_values
    gsql1_total_diff[count] = np.linalg.norm(norm_diff_gsql1,axis =1)
    time_gsql1_end=time.time()
    
    #GSQL2
    time_gsql2_st=time.time()
    ql3 = mdptoolbox.mdp.GSQL2(P, R, discount,n_iter=iterations)
    ql3.run()
    norm_diff_gsql2 = vi.V - ql3.q_values
    gsql2_total_diff[count] = np.linalg.norm(norm_diff_gsql2,axis =1)
    time_gsql2_end=time.time()

    # #ZAPQL
    # time_zap_st=time.time()
    # zapql = mdptoolbox.mdp.ZAPQLearning(P, R, discount,alpha=0.1, epsilon=0.1, n_iter=iterations, zeta=0.1)
    # zapql.run()
    # norm_diff_zapql = vi.V - zapql.q_values
    # zapql_total_diff[count] = np.linalg.norm(norm_diff_zapql,axis =1)
    # time_zap_end=time.time()

    # #Double QL
    time_dql_st=time.time()
    dql = mdptoolbox.mdp.DoubleQLearning(P, R, discount,n_iter=iterations)
    dql.run()
    norm_diff_dql = vi.V - dql.q_values
    dql_total_diff[count] = np.linalg.norm(norm_diff_dql,axis =1)
    time_dql_end=time.time()

    #GSQL3_SPSA
    time_gsql3_st=time.time()
    if(count%2==0):
        ql4 = mdptoolbox.mdp.GSQL3(P, R, discount,max(1,min((w+delta),20)),n_iter=iterations)
        ql4.run()
        norm_diff_gsql3 = vi.V - ql4.q_values
        gsql3_total_diff[count] = np.linalg.norm(norm_diff_gsql3,axis =1)
        # print("gsql3_total_diff[count]",gsql3_total_diff[count])
        err=np.mean(gsql3_total_diff[count])
        # print("error",err)
        w=w-(w_update_lr*(err/delta))
    else:
        ql4 = mdptoolbox.mdp.GSQL3(P, R, discount,max(1,min((w-delta),20)),n_iter=iterations)
        ql4.run()
        norm_diff_gsql3 = vi.V - ql4.q_values
        gsql3_total_diff[count] = np.linalg.norm(norm_diff_gsql3,axis =1)
        # print("gsql3_total_diff[count]",gsql3_total_diff[count])
        err=np.mean(gsql3_total_diff[count])
        # print("error",err)
        w=w+(w_update_lr*(err/delta))
    w_val.append(w)
    # print("value of w:",w)
    time_gsql3_end=time.time()
    if count==0:
        time_dql=time_dql_end-time_dql_st
        time_sor_original=time_sor_original_end-time_sor_original_st
        time_sql=time_sql_end-time_sql_st
        time_gsql1=time_gsql1_end - time_gsql1_st
        time_gsql2=time_gsql2_end - time_gsql2_st
        time_gsql3=time_gsql3_end - time_gsql3_st
        # time_zap=time_zap_end-time_zap_st
        np.savetxt("time_compare_algo_100.csv", np.c_[time_dql,time_sor_original,time_sql,time_gsql1,time_gsql2,time_gsql3], delimiter=",")

np.savetxt('w_val_GSQL_SPSA_20.csv', [w_val], delimiter=',', fmt='%f')

#Calculate average error and standard deviation   
avg_error_dql=np.mean(dql_total_diff,axis=0)
std_error_dql=np.std(dql_total_diff,axis=0)

avg_error_sor=np.mean(sor_total_diff,axis=0)
std_error_sor=np.std(sor_total_diff,axis=0)

avg_error_sql=np.mean(sql_total_diff,axis=0)
std_error_sql=np.std(sql_total_diff,axis=0)

avg_error_gsql1=np.mean(gsql1_total_diff,axis=0)
std_error_gsql1=np.std(gsql1_total_diff,axis=0)

avg_error_gsql2=np.mean(gsql2_total_diff,axis=0)
std_error_gsql2=np.std(gsql2_total_diff,axis=0)

avg_error_gsql3=np.mean(gsql3_total_diff,axis=0)
std_error_gsql3=np.std(gsql3_total_diff,axis=0)

# avg_error_zapql=np.mean(zapql_total_diff,axis=0)
# std_error_zapql=np.std(zapql_total_diff,axis=0)

np.savetxt("mean_std_compare_algo_100.csv", np.c_[avg_error_dql,std_error_dql,avg_error_sor,std_error_sor,avg_error_sql,std_error_sql,avg_error_gsql1,std_error_gsql1,avg_error_gsql2,std_error_gsql2,avg_error_gsql3,std_error_gsql3], delimiter=",")


0
1
2
3
4
5
6
7
8
9


In [7]:
##Zap-QL
s = 100
a = 5
discount = 0.6
delta=0.6

episodes = 2
iterations = 100000

zapql_total_diff=np.zeros((episodes,iterations))
for count in range(episodes):
 #ZAPQL
    time_zap_st=time.time()
    zapql = mdptoolbox.mdp.ZAPQLearning(P, R, discount,alpha=0.1, epsilon=0.1, n_iter=iterations, zeta=0.1)
    zapql.run()
    norm_diff_zapql = vi.V - zapql.q_values
    zapql_total_diff[count] = np.linalg.norm(norm_diff_zapql,axis =1)
    time_zap_end=time.time()
    if count==0:
        print("zap_ql time", time_zap_end-time_zap_st)
avg_error_zapql=np.mean(zapql_total_diff,axis=0)
std_error_zapql=np.std(zapql_total_diff,axis=0) 


np.savetxt("mean_std_zap_ql_100.csv", np.c_[avg_error_zapql,std_error_zapql], delimiter=",")

zap_ql time 6473.75822520256
